In [1]:
import tensorflow as tf
print("NUM GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

NUM GPUs Available:  0


In [12]:
# data processing
# x_train is like past n days, y_train is future n days
# past n days will be 365 ? future n days is 7 ? for the week?
import pickle
import numpy as np
import pandas as pd
full_set = pd.DataFrame()
len_test = 0
start_year = 2017
end_year = 2022
for x in range(start_year, end_year+1):
    filepath = "data/" + str(x) + "-wozone.pkl"
    infile2 = open(filepath, 'rb')
    ozone_data = pickle.load(infile2)
    print(ozone_data.head())
    new_df = pd.DataFrame(ozone_data[['AQI', 'Ozone']])
    full_set = full_set.append(new_df)
    len_test = new_df.shape[0]
training_set = full_set.iloc[:-len_test, :]
test_set = full_set.iloc[-len_test:, :]


        Date     Ozone   AQI
0 2017-02-28  0.030000  43.0
1 2017-03-01  0.032510  37.0
2 2017-03-02  0.040834  40.0
3 2017-03-03  0.036880  37.0
4 2017-03-04  0.038213  41.0
        Date     Ozone   AQI
0 2018-02-28  0.022000  23.0
1 2018-03-01  0.021315  36.0
2 2018-03-02  0.042683  41.0
3 2018-03-03  0.043745  43.0
4 2018-03-04  0.043759  43.0


<ipython-input-12-081032f91e4f>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_set = full_set.append(new_df)
<ipython-input-12-081032f91e4f>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_set = full_set.append(new_df)


In [53]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
training_set_scaled = sc.fit_transform(training_set)

x_train = []
y_train = []
n_future = 1 # next 1 day of temperature forecast
n_past = 30 # Past 30 days
for i in range(0,len(training_set_scaled)-n_past-n_future+1):
    x_train.append(training_set_scaled[i : i + n_past])
    y_train.append(training_set_scaled[i + n_past : i + n_past + n_future, 0])
x_train , y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0] , x_train.shape[1], 2) )
y_train = np.reshape(y_train, (y_train.shape[0], )  )

In [ ]:
# data visualization

In [ ]:
# data standardizing

In [25]:
# model: recurrent neural network

# x_train and y_train should be numpy arrays

# https://medium.com/analytics-vidhya/weather-forecasting-with-recurrent-neural-networks-1eaa057d70c3

from keras.models import Sequential
from keras.layers import Bidirectional, LSTM,Dense ,Dropout


In [54]:
# set up regressor model
regressor = Sequential()
regressor.add(Bidirectional(LSTM(units=30, return_sequences=True, input_shape = (x_train.shape[1],2) ) ))
#regressor.add(LSTM(units=30, return_sequences=True, input_shape = (x_train.shape[1],x_train.shape[2]) ) )
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 30 , return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 30 , return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 30))
regressor.add(Dropout(0.2))
regressor.add(Dense(units = n_future,activation='linear'))
#regressor.add(Dense(units = 1,activation='linear'))
regressor.compile(optimizer='adam', loss='mean_squared_error',metrics=['acc']) # adam optimizer for optimal efficiency
regressor.fit(x_train, y_train, epochs=500,batch_size=32 )
#history = regressor.fit(x_train, y_train, epochs=50, batch_size=32)
# plot history
#pyplot.plot(history.history['loss'], label='train')
#pyplot.plot(history.history['val_loss'], label='test')
#pyplot.legend()
#pyplot.show()


Epoch 1/500
7/7 [==============================] - 16s 82ms/step - loss: nan - acc: 0.0046
Epoch 2/500
7/7 [==============================] - 1s 81ms/step - loss: nan - acc: 0.0046
Epoch 3/500
7/7 [==============================] - 1s 74ms/step - loss: nan - acc: 0.0046
Epoch 4/500
7/7 [==============================] - 1s 79ms/step - loss: nan - acc: 0.0046
Epoch 5/500
1/7 [===>..........................] - ETA: 0s - loss: nan - acc: 0.0000e+00

KeyboardInterrupt: ignored

In [55]:
print(x_train.shape)
print(y_train.shape)

(216, 30, 2)
(216,)


In [ ]:
# read test dataset
testdataset = test_set
real_temperature = test_set
testing = sc.transform(testdataset)
testing = np.array(testing)
testing = np.reshape(testing,(testing.shape[1],testing.shape[0],1))

# predict model
predicted_temperature = regressor.predict(testing)
predicted_temperature = sc.inverse_transform(predicted_temperature)
predicted_temperature = np.reshape(predicted_temperature,(predicted_temperature.shape[1],predicted_temperature.shape[0]))

print(predicted_temperature)

1/1 [==============================] - 0s 24ms/step
[[23.847404]
 [21.79368 ]
 [19.517614]
 [21.689926]]
